In [212]:
import numpy as np
import pandas as pd

In [218]:
X_train = pd.read_csv("../Data/df_train_vvfinal").drop("Unnamed: 0", axis = 1)
y_train = pd.read_csv("../Data/y_train_vfinal").drop("Unnamed: 0", axis = 1)

In [219]:
X_val = pd.read_csv("../Data/df_val_vvfinal").drop("Unnamed: 0", axis = 1)
y_val = pd.read_csv("../Data/y_val_vfinal").drop("Unnamed: 0", axis = 1)

In [220]:
X_test = pd.read_csv("../Data/df_test_vvfinal").drop("Unnamed: 0", axis = 1)
y_test = pd.read_csv("../Data/y_test_vfinal").drop("Unnamed: 0", axis = 1)

In [221]:
X_train_normalize = (X_train - X_train.mean())/X_train.std()
y_train_normalize = (y_train - y_train.mean())/y_train.std()

In [171]:
y_train_normalize = (y_train - y_train.mean())/y_train.std()

In [172]:
y_train_normalize

,price_doc
0,-0.265169
1,-0.234011
2,-0.296328
3,1.240843
4,0.409940
...,...
27419,0.056806
27420,3.712780
27421,-0.032317
27422,1.323933


## Neural Network

In [204]:
class NeuralNetwork(): 
    def fit(self, X, y, n_hidden, nodes, activations, lr, batch_size = 0):
        self._lr = lr
        self._X = X.values
        self._y = y.values
        self._n_hidden = n_hidden
        self._nodes = nodes
        self._weights = self._generate_weights()
        self._biases = self._generate_bias()
        self._activations = activations
        self._forward_inputs = []
        self._batch_size = batch_size

        
        self._train()
        
    def _activation(self, data, activation = "relu"):
        if activation == "relu":
            def relu(data):
                return np.array([max(0,i) for i in data]).reshape(data.shape)
            return np.apply_along_axis(relu, 1, data)
        if activation == "sigmoid":
            def sigmoid(data):
                return (1/(1 + np.exp(-data))).reshape(data.shape)
            return np.apply_along_axis(sigmoid, 1, data)
    
    def _der_activation(self, points, activation = "relu"):
        if activation == "relu":
            def d_relu(point):
                return np.array([0 if y <= 0 else 1 for y in point])
            return np.apply_along_axis(d_relu, 1, points)
        if activation == "sigmoid":
            ## todo
            return
    
    def _loss_function(self, ypred, loss = "l2"):
        if self._batch_size > 0:
            y = self._batchy
        else:
            y = self._y
        if loss == "mse":
            return ((ypred - y) ** 2).mean()
        if loss == "l2":
            return (((ypred - y) ** 2)/2).mean()
    
    def _loss_jacobian(self, ypred, loss = "l2"):
        if self._batch_size > 0:
            y = self._batchy
        else:
            y = self._y
        if loss == "l2":
            return (ypred - y)/(len(ypred))
    
    def _generate_weights(self):
        hidden_weights = []
        nodes = self._nodes
        for idx in range(1,len(nodes)):
            hidden_weights.append(0.01 *np.random.randn(nodes[idx -1], nodes[idx]))

        return hidden_weights
    
    def _generate_bias(self):
        hidden_layers = []
        nodes = self._nodes
        for i in range(self._n_hidden + 1):
            hidden_layers.append(np.zeros((nodes[i + 1], 1)))
        return hidden_layers
    
    
    def _forward_propagation(self):
        """
        Suppose 2 observations
        
        Suppose previous layer is 3 nodes
        Suppose current layer is 2 nodes
        
        prev shape (2,3)
        prev = ob1 [prev_node_1 val, prev_node_2 val, prev_node_3 val]
               ob2 [prev_node_1 val, prev_node_2 val, prev_node_3 val]
               
        layer shape (3,2)
        layer = [weight for current_node_1 for prev_node_1, weight for current_node_2 for prev_node_1]
                [weight for current_node_1 for prev_node_2, weight for current_node_2 for prev_node_2]
                [weight for current_node_1 for prev_node_3, weight for current_node_2 for prev_node_3]
                
        output shape (2,2) # since 2 observations and 2 layers
        output = ob1 [current_node_1 val, current_node_2 val]
                 ob2 [current_node_1 val, current_node_2 val]
                 
        Then for bias in current layer it is (2,1) since 2 nodes in current layer
        
        So for each row in output we add the bias row wise and apply the activation function to each row
        
        prev <- ouput
        
        Move onto next layer...
        """
        if self._batch_size > 0:
            prev = self._batchX
        else:
            prev = self._X
        weights = self._weights
        biases = self._biases
        activations = self._activations[1:-1]
    
        for idx, layer in enumerate(weights):
            if idx == (len(weights) - 1):
                self._forward_inputs.append((prev, None))
                prev = (prev @ layer) + biases[idx].T,
            else:
                weight_output = (prev @ layer) + biases[idx].T
                self._forward_inputs.append((prev, weight_output))
                prev = self._activation(data = weight_output, activation = activations[idx])

        return prev
    
    def _backward_propagation(self, ypred):
            
        j = self._loss_jacobian(ypred)
        
        temp_weights = self._weights
        temp_biases = self._biases
                
        for i in range(len(self._forward_inputs)-1, -1, -1):
            if i != (len(self._forward_inputs) - 1):
                # activation func on all layers except the last
                der_acti = self._der_activation(self._forward_inputs[i][1])
                j = np.multiply(j,der_acti)

            x = self._forward_inputs[i][0]

            jw = x.T.dot(j)

            b = np.ones((j.shape[0],1))
            jb = j.T.dot(b)
            
            j = j.dot(temp_weights[i].T)
            
            temp_weights[i] -= self._lr * jw
            temp_biases[i] -= self._lr * jb

            #j = j.dot(self._weights[i].T)
            
        self._forward_inputs = []
        return temp_weights, temp_biases
        #self._forward_inputs = []

        
    
    def _train(self):
        min_loss = old_loss = np.inf
        losses = []
        mses = []
        tol = 0.00001
        terminate_count = anneal_count = 0
        while True:
            if self._batch_size > 0:
                X_index = np.arange(self._X.shape[0])
                np.random.shuffle(X_index)
                batch_index = X_index[:self._batch_size]
                self._batchX = self._X[batch_index,:]
                self._batchy = self._y[batch_index,:]
            
            batched_out = self._forward_propagation()
            validation_out = self.predict(self._val_X)
                
            loss = self._loss_function(validation_out)
            mse = self._loss_function(validation_out, loss = "mse")
            print("\nloss:")
            print(loss)
            print("mse:")
            print(mse)
            if loss <= min_loss:
                min_loss = loss
                terminate_count = anneal_count = 0
            if loss <= old_loss:
                 anneal_count = 0
                print("INCREASE IN LOSS")
            else:
                terminate_count += 1
                anneal_count += 1
                print("INCREASE IN LOSS")
                if anneal_count >= 2:
                    anneal_count = 0
                    self._lr = self._lr / 2
                    print("Decreasing learning rate. New rate is " + str(self._lr))
                if terminate_count > 20 or self._lr < tol:
                    break
            
            losses.append(loss)
            mses.append(mse)
            self._backward_propagation(batched_out[0])
            old_loss = loss
            
        return losses, mses

In [205]:
INPUT_SIZE = X_train_normalize.shape[1]
OUTPUT_SIZE = 1
LEARNING_RATE = 0.001
nodes = [INPUT_SIZE,100,OUTPUT_SIZE]
activations = ["relu" for i in range(len(nodes))]

nn = NeuralNetwork()

nn.fit(X = X_train_normalize,
       y = y_train_normalize,
       n_hidden = len(nodes) - 2,
       nodes = nodes,
       activations = activations,
       lr = LEARNING_RATE,
       batch_size = 100)


loss:
0.29150017812800816
mse:
0.5830003562560163

loss:
0.358792220569141
mse:
0.717584441138282
Decreasing learning rate. New rate is 0.0005

loss:
0.732827956384105
mse:
1.46565591276821
Decreasing learning rate. New rate is 0.00025

loss:
0.7618701255668673
mse:
1.5237402511337346
Decreasing learning rate. New rate is 0.000125

loss:
0.19493546797546768
mse:
0.38987093595093536

loss:
0.4308129147019706
mse:
0.8616258294039412
Decreasing learning rate. New rate is 6.25e-05

loss:
0.32964180108548824
mse:
0.6592836021709765
Decreasing learning rate. New rate is 3.125e-05

loss:
0.20810361906382543
mse:
0.41620723812765087
Decreasing learning rate. New rate is 1.5625e-05

loss:
0.261769710707261
mse:
0.523539421414522
Decreasing learning rate. New rate is 7.8125e-06
